In [6]:
import torch 
import torch.nn as nn

from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from utils import *
from tqdm import tqdm

dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
d = 3
m = 8
t = 200
n = 20000
snr_min = 0
snr_max = 20
lamda = 0.2
radius = 0.1
mc_range = 3
coherent = False

array = UCA(m, lamda)
array.build_array(radius=0.1)
array.build_array_manifold()
array.build_transform_matrices_from_array_manifold()

observations, angles = generate_data(n, t, d, snr_min, snr_max, array, mc_range, coherent)

In [8]:
lr = 1e-2
wd = 1e-8
nbEpoches = 200
batchSize = 256

x_train, x_valid, theta_train, theta_valid = train_test_split(observations, angles, test_size=0.2)
x_train, x_test, theta_train, theta_test = train_test_split(x_train, theta_train, test_size=0.2)

train_set = DATASET(x_train, theta_train)
valid_set = DATASET(x_valid, theta_valid)
test_set = DATASET(x_test, theta_test)

train_loader = DataLoader(train_set, batch_size=batchSize, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batchSize, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batchSize, shuffle=False)

In [9]:
class my_model(nn.Module):

    def __init__(self, m: int, d: int, mc_range: int, array):
        
        super().__init__()
        self.m = m
        self.d = d
        self.array = array

        self.W = array.transform_matrices[..., :m-d-1].to(dev)

        self.bn = nn.BatchNorm1d(2*self.m)
        self.rnn = nn.GRU(input_size=2*self.m, hidden_size=2*self.m, num_layers=1, batch_first=True)
        self.fc = nn.Linear(in_features=2*self.m, out_features=2*self.m*self.m)
        self.mlp = nn.Sequential(nn.Linear(in_features=array.nbSamples_spectrum, out_features=2*self.m), nn.ReLU(),
                                 nn.Linear(in_features=2*self.m, out_features=2*self.m), nn.ReLU(),
                                 nn.Linear(in_features=2*self.m, out_features=2*self.m), nn.ReLU(),
                                 nn.Linear(in_features=2*self.m, out_features=self.d))

    def forward(self, X: torch.Tensor):
        
        X = torch.cat((torch.real(X), torch.imag(X)), dim=-1)
        X = self.bn(X.transpose(1, 2)).transpose(1, 2)
        _, X = self.rnn(X)

        cov = self.fc(X[-1])
        cov = cov.reshape(-1, 2, self.m, self.m)
        cov = cov[:, 0, :, :] + 1j * cov[:, 1, :, :]
        cov = cov @ cov.conj().transpose(1, 2)
        vals, vecs = torch.linalg.eigh(cov)
        idx = torch.sort(torch.abs(vals), dim=1)[1].unsqueeze(dim=1).repeat(repeats=(1, self.m, 1))
        vecs = torch.gather(vecs, dim=2, index=idx)
        En = vecs[:, :, :(self.m - self.d)]
        
        Q = torch.einsum('smr,bmk->bsrk', self.W.conj(), En)
        Q = Q @ Q.conj().transpose(-2, -1)
        # spectrum = 1 / torch.abs(torch.linalg.det(Q))
        spectrum = 1 / torch.linalg.eigvalsh(Q)[..., 0]

        theta = self.mlp(spectrum)
        
        return theta

In [10]:
model = my_model(m, d, mc_range, array).to(dev)
# model.load_state_dict(torch.load('model_1_0dB.pt', weights_only=True))
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.2)
Loss, Loss_theta, Val_theta = [], [], []
bestVal = 1000

for i in tqdm(range(nbEpoches)):
    # Train
    running_loss = 0.0
    for data in train_loader:
        X, theta_true = data[0].to(dev), data[1].to(dev)
        optimizer.zero_grad()
        theta_pred = model(X)
        loss = RMSPE(theta_pred, theta_true) 
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    Loss.append(running_loss/len(train_loader))

    # scheduler.step()

    # Validation 
    with torch.no_grad():
        running_loss = 0.0
        for data in valid_loader:
            X, theta_true = data[0].to(dev), data[1].to(dev)
            theta_pred = model(X)
            loss = RMSPE(theta_pred, theta_true)
            running_loss += loss.item()
        
        Val_theta.append(running_loss/len(test_loader))

        if Val_theta[i] < bestVal:
            bestVal = Val_theta[i]
            torch.save(model.state_dict(), 'model_1.pt')

    print("Iteration {}: Loss theta = {}".format(i, Loss[-1]))

  0%|          | 1/200 [00:03<11:02,  3.33s/it]

Iteration 0: Loss theta = 0.5100754737854004


  1%|          | 2/200 [00:06<10:58,  3.33s/it]

Iteration 1: Loss theta = 0.48576536774635315


  2%|▏         | 3/200 [00:09<10:55,  3.33s/it]

...

Iteration 197: Loss theta = 0.1497730404138565


100%|█████████▉| 199/200 [10:30<00:02,  2.91s/it]

Iteration 198: Loss theta = 0.15123320788145064


100%|██████████| 200/200 [10:33<00:00,  3.17s/it]

Iteration 199: Loss theta = 0.14964014589786528


In [11]:
model_test = my_model(m, d, mc_range, array).to(dev)
model_test.load_state_dict(torch.load('model_1.pt', weights_only=True))
running_loss = 0.0 

with torch.no_grad():
    for data in test_loader:
        X, theta_true= data[0].to(dev), data[1].to(dev)
        theta_pred = model_test(X)
        loss = RMSPE(theta_pred, theta_true)
        running_loss += loss.item()

Acc_theta = running_loss/len(test_loader)

print("Mixed RMSE DoA =", Acc_theta)

Mixed RMSE DoA = 0.14915905892848969


In [12]:
observations, angles = generate_data(100, t, d, 0, 0, array, mc_range, coherent)
Acc_theta_0 = RMSPE(model_test(observations.to(dev)), angles)

print("0dB RMSE DoA =", Acc_theta_0)


angles_rare = []
for i in range(observations.shape[0]):
    angles_rare.append(RARE(observations[i].T, d, 3, array)[0])
angles_rare = torch.stack(angles_rare, dim=0)

print("RMSPE RARE =", RMSPE(angles_rare.cuda(), angles.cuda()))

0dB RMSE DoA = tensor(0.1903, device='cuda:0', grad_fn=<MeanBackward0>)
RMSPE RARE = tensor(0.2966, device='cuda:0')


In [13]:
observations, angles = generate_data(100, t, d, 5, 5, array, mc_range, coherent)
Acc_theta_5 = RMSPE(model_test(observations.to(dev)), angles)

print("5dB RMSE DoA =", Acc_theta_5)

angles_rare = []
for i in range(observations.shape[0]):
    angles_rare.append(RARE(observations[i].T, d, 3, array)[0])
angles_rare = torch.stack(angles_rare, dim=0)

print("RMSPE RARE =", RMSPE(angles_rare.cuda(), angles.cuda()))

5dB RMSE DoA = tensor(0.1567, device='cuda:0', grad_fn=<MeanBackward0>)
RMSPE RARE = tensor(0.2387, device='cuda:0')


In [14]:
observations, angles = generate_data(100, t, d, 10, 10, array, mc_range, coherent)
Acc_theta_10 = RMSPE(model_test(observations.to(dev)), angles)

print("10dB RMSE DoA =", Acc_theta_10)

angles_rare = []
for i in range(observations.shape[0]):
    angles_rare.append(RARE(observations[i].T, d, 3, array)[0])
angles_rare = torch.stack(angles_rare, dim=0)

print("RMSPE RARE =", RMSPE(angles_rare.cuda(), angles.cuda()))

10dB RMSE DoA = tensor(0.1546, device='cuda:0', grad_fn=<MeanBackward0>)
RMSPE RARE = tensor(0.1963, device='cuda:0')


In [15]:
observations, angles = generate_data(100, t, d, 15, 15, array, mc_range, coherent)
Acc_theta_15 = RMSPE(model_test(observations.to(dev)), angles)

print("15dB RMSE DoA =", Acc_theta_15)

angles_rare = []
for i in range(observations.shape[0]):
    angles_rare.append(RARE(observations[i].T, d, 3, array)[0])
angles_rare = torch.stack(angles_rare, dim=0)

print("RMSPE RARE =", RMSPE(angles_rare.cuda(), angles.cuda()))

15dB RMSE DoA = tensor(0.1365, device='cuda:0', grad_fn=<MeanBackward0>)
RMSPE RARE = tensor(0.1125, device='cuda:0')


In [16]:
observations, angles = generate_data(100, t, d, 20, 20, array, mc_range, coherent)
Acc_theta_20 = RMSPE(model_test(observations.to(dev)), angles)

print("20dB RMSE DoA =", Acc_theta_20)

angles_rare = []
for i in range(observations.shape[0]):
    angles_rare.append(RARE(observations[i].T, d, 3, array)[0])
angles_rare = torch.stack(angles_rare, dim=0)

print("RMSPE RARE =", RMSPE(angles_rare.cuda(), angles.cuda()))

20dB RMSE DoA = tensor(0.1310, device='cuda:0', grad_fn=<MeanBackward0>)
RMSPE RARE = tensor(0.0299, device='cuda:0')
